# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [1]:
import os
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

from keras.preprocessing.image import ImageDataGenerator
#
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras import optimizers
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import load_model
from keras.models import Model

from PIL import Image

Using TensorFlow backend.


In [2]:
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Hi Tim, vorrangig aus GPU-Gründen haben wir die Jan(n)is-Gruppe bei dieser Übung als auch bei den nächsten beiden mit ins Boot geholt.")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Hi Tim, vorrangig aus GPU-Gründen haben wir die Jan(n)is-Gruppe bei dieser Übung als auch bei den nächsten beiden mit ins Boot geholt.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [3]:
batch_size = 32
image_input_size = (224, 224)
data_path = './data/'

prob_drop_conv = 0.5              
pool_size = (2, 2) 
nb_epoch = 150
opt = optimizers.RMSprop(lr=0.000001, rho=0.9)

In [4]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

izw_classes = ('unknown', 'cheetah', 'leopard')

generator = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
val_generator = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False)

train_gen = generator.flow_from_directory(
    train_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

val_gen = val_generator.flow_from_directory(
    val_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)


Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [5]:

#das convolutional net tutorial von keras

# Convolutional model
model = Sequential()
steps_per_epoch=len(pool_size)/batch_size

# conv1 layer
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu', input_shape=[224,224,3]))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(625, activation='relu'))
model.add(Dropout(prob_drop_conv))

# fc2 layer
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit_generator(train_gen,steps_per_epoch, nb_epoch=nb_epoch, shuffle=True, verbose=1)



W0815 17:07:03.143175 4509324736 deprecation_wrapper.py:119] From /Users/ced/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/Users/ced/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", input_shape=[224, 224,..., padding="same")`
  
W0815 17:07:03.145573 4509324736 deprecation_wrapper.py:119] From /Users/ced/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 17:07:03.149528 4509324736 deprecation_wrapper.py:119] From /Users/ced/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/Users/ced/miniconda3/lib/python3.7/site-packag

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 4)         580       
__________

Epoch 13/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 341ms/step - loss: 15.1107 - acc: 0.0625
Epoch 14/150
1/0 [====================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 372ms/step - loss: 13.6082 - acc: 0.1562
Epoch 29/150
1/0 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Epoch 44/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 349ms/step - loss: 14.5927 - acc: 0.0938
Epoch 45/150
1/0 [====================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 1s 511ms/step - loss: 11.5315 - acc: 0.2500
Epoch 91/150
1/0 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 338ms/step - loss: 11.5966 - acc: 0.2500
Epoch 106/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 353ms/step - loss: 13.6014 - acc: 0.1562
Epoch 121/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 379ms/step - loss: 13.1521 - acc: 0.1562
Epoch 136/150
1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1/0 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 384ms/step - loss: 12.0961 - acc: 0.2500


In [6]:
# Evaluate
#evaluation = model.evaluate_generator(val_gen, verbose=1)
evaluation = model.evaluate_generator(val_gen, 1)

print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

Summary: Loss over the test dataset: 14.62, Accuracy: 0.06


Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [7]:
# TODO

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [8]:
model2 = ResNet50(weights = 'imagenet')
model2.summary()

W0815 17:08:01.007290 4509324736 deprecation_wrapper.py:119] From /Users/ced/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:
evaluation2 = model2.predict_generator(val_gen, steps=(len(val_gen)), verbose=1)

60/60 [==============================] - 174s 3s/step


In [10]:
#TODO DELETE THIS
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
pred = decode_predictions(evaluation2, top=1)
print('Predicted:', pred)


Predicted: [[('n02114855', 'coyote', 0.4485118)], [('n03788365', 'mosquito_net', 0.2686611)], [('n03891251', 'park_bench', 0.10064046)], [('n04371774', 'swing', 0.22957754)], [('n02114855', 'coyote', 0.45637178)], [('n03388043', 'fountain', 0.97615653)], [('n03891251', 'park_bench', 0.113026984)], [('n02130308', 'cheetah', 0.91280824)], [('n03388043', 'fountain', 0.18641663)], [('n03388043', 'fountain', 0.96286756)], [('n03388043', 'fountain', 0.97800064)], [('n03743016', 'megalith', 0.46531516)], [('n15075141', 'toilet_tissue', 0.63400984)], [('n02130308', 'cheetah', 0.36803973)], [('n03788365', 'mosquito_net', 0.33781677)], [('n03388043', 'fountain', 0.63867)], [('n03388043', 'fountain', 0.7120134)], [('n02125311', 'cougar', 0.16781376)], [('n03388043', 'fountain', 0.2860591)], [('n09193705', 'alp', 0.24350822)], [('n02130308', 'cheetah', 0.17853816)], [('n01877812', 'wallaby', 0.428212)], [('n02130308', 'cheetah', 0.8734413)], [('n02391049', 'zebra', 0.46828696)], [('n02130308', 'ch

In [12]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

                                                                 activation_16[0][0]              
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 28, 28, 512)  0           add_6[0][0]                      
__________________________________________________________________________________________________
res3d_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_19[0][0]              
__________________________________________________________________________________________________
bn3d_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 28, 28, 128)  0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
res3d_bran

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [13]:
for img in pred:
    if img[0][1] not in izw_classes:
        img[0] = (img[0][0],'unknown',img[0][2])

for img in pred:
    print(img)

[('n02114855', 'unknown', 0.4485118)]
[('n03788365', 'unknown', 0.2686611)]
[('n03891251', 'unknown', 0.10064046)]
[('n04371774', 'unknown', 0.22957754)]
[('n02114855', 'unknown', 0.45637178)]
[('n03388043', 'unknown', 0.97615653)]
[('n03891251', 'unknown', 0.113026984)]
[('n02130308', 'cheetah', 0.91280824)]
[('n03388043', 'unknown', 0.18641663)]
[('n03388043', 'unknown', 0.96286756)]
[('n03388043', 'unknown', 0.97800064)]
[('n03743016', 'unknown', 0.46531516)]
[('n15075141', 'unknown', 0.63400984)]
[('n02130308', 'cheetah', 0.36803973)]
[('n03788365', 'unknown', 0.33781677)]
[('n03388043', 'unknown', 0.63867)]
[('n03388043', 'unknown', 0.7120134)]
[('n02125311', 'unknown', 0.16781376)]
[('n03388043', 'unknown', 0.2860591)]
[('n09193705', 'unknown', 0.24350822)]
[('n02130308', 'cheetah', 0.17853816)]
[('n01877812', 'unknown', 0.428212)]
[('n02130308', 'cheetah', 0.8734413)]
[('n02391049', 'unknown', 0.46828696)]
[('n02130308', 'cheetah', 0.9901253)]
[('n04371774', 'unknown', 0.1586055

[('n03388043', 'unknown', 0.75624245)]
[('n03160309', 'unknown', 0.15382472)]
[('n03388043', 'unknown', 0.5004064)]
[('n02115641', 'unknown', 0.1613866)]
[('n02130308', 'cheetah', 0.95393926)]
[('n02480855', 'unknown', 0.14123948)]
[('n03388043', 'unknown', 0.9866181)]
[('n03388043', 'unknown', 0.2841917)]
[('n03388043', 'unknown', 0.88548064)]
[('n02130308', 'cheetah', 0.44841567)]
[('n03388043', 'unknown', 0.27538982)]
[('n04589890', 'unknown', 0.38828263)]
[('n02391049', 'unknown', 0.28611243)]
[('n02130308', 'cheetah', 0.5822802)]
[('n02808440', 'unknown', 0.36215526)]
[('n02130308', 'cheetah', 0.4970743)]
[('n03388043', 'unknown', 0.54585844)]
[('n03388043', 'unknown', 0.7613267)]
[('n02130308', 'cheetah', 0.5619429)]
[('n02965783', 'unknown', 0.1148753)]
[('n03388043', 'unknown', 0.80835766)]
[('n02130308', 'cheetah', 0.5508322)]
[('n03388043', 'unknown', 0.855549)]
[('n02130308', 'cheetah', 0.75662607)]
[('n03388043', 'unknown', 0.25432044)]
[('n02130308', 'cheetah', 0.8133302)]

[('n02423022', 'unknown', 0.1889775)]
[('n03388043', 'unknown', 0.64352185)]
[('n03788365', 'unknown', 0.86886436)]
[('n03388043', 'unknown', 0.4929051)]
[('n02130308', 'cheetah', 0.99204105)]
[('n04589890', 'unknown', 0.19430083)]
[('n03388043', 'unknown', 0.9125776)]
[('n02110341', 'unknown', 0.104983896)]
[('n04371774', 'unknown', 0.17962627)]
[('n02130308', 'cheetah', 0.97166765)]
[('n02110341', 'unknown', 0.939961)]
[('n03498962', 'unknown', 0.08375412)]
[('n02130308', 'cheetah', 0.9659305)]
[('n03891251', 'unknown', 0.08400569)]
[('n02130308', 'cheetah', 0.9032757)]
[('n03743016', 'unknown', 0.22741283)]
[('n03388043', 'unknown', 0.3474213)]
[('n02437616', 'unknown', 0.42894307)]
[('n02965783', 'unknown', 0.25510183)]
[('n02504013', 'unknown', 0.08797813)]
[('n03388043', 'unknown', 0.9988194)]
[('n03388043', 'unknown', 0.22233868)]
[('n03388043', 'unknown', 0.49637896)]
[('n02130308', 'cheetah', 0.93314314)]
[('n02130308', 'cheetah', 0.7831782)]
[('n03788365', 'unknown', 0.561587

## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [14]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_33[0][0]              
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_11 (Add)                    (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
                                                                 activation_31[0][0]              
__________________________________________________________________________________________________
activation_34 (Activation)      (None, 14, 14, 1024) 0           add_11[0][0]                     
__________________________________________________________________________________________________
res4e_branch2a (Conv2D)         (None, 14, 14, 256)  262400      activation_34[0][0]              
__________

In [15]:
for layer in model2.layers:
    layer.trainable = False
    print(layer)
model2.layers.pop()
print(model2.layers.pop())
model2.layers.pop()
model2.summary()

model2.layers[-1].outbound_nodes = []
model2.outputs = [model2.layers[-1].output]
output1 = model2.get_layer('avg_pool').output
output2 = Flatten()(output2)
output3 = Dense(output_dim=3)(output3)
model3 = Model(model2.input,output3)
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

ValueError: No such layer: avg_pool

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [16]:
model3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model3.fit_generator(train_gen, steps_per_epoch=100, nb_epoch=nb_epoch, shuffle=True, verbose=1)

NameError: name 'model3' is not defined

# Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?